# 이미지 이상탐지모델(GAN)

- 공항 출국장에서 수하물 X-선 검사를 받으면서 보안검사를 받아보신 경험들이 한번쯤 있으시겠죠? 실은 비단 여러분들의 수하물 정도가 아니라 비행기에 실리고 내리는 모든 화물들에 대해 보이지 않는 어딘가에서 X-선으로 꼼꼼이 검사를 하고 있답니다. 그러기 위해 보안 검사원들이 불철주야 수고를 하고 계신데요, 블랙프라이데이 같은 시즌이 도래하면 하루에도 똑같은 가방 수천개가 컨베이어벨트를 지나가는 것을 눈을 떼지 못하고 X-선으로 지켜보고 있어야만 하는 세관 검사원들의 웃지못할 이야기도 있다고 합니다.

- 딥러닝 기술이 유용하게 사용될 수 있는 대표적인 사례로 바로 이상감지(Anomaly Detection)를 꼽을 수 있습니다. 위 그림에서처럼 혹시라도 위험한 물품이 포함되어 있지 않을까 24시간 사람이 뚫어져라 모니터를 쳐다보고 있어야만 하는 고된 일을 인공지능이 대신 정확하게 수행해 줄 수 있다면 좋겠죠? 이런 과제는 실제로 공항, 항만, 주요 군사시설 등에서 매우 관심을 가지고 진행하고 있답니다. 그런데 이런 이상감지 모델로 GAN 기술이 유용하게 활용되고 있다고 합니다.

- 왜 그럴까요? 그냥 쉽게 생각할 수 있는 지도학습(supervised learning)으로는 어려울까요? 네 그렇습니다. 이상감지 태스크의 경우, 정상(Normal) 상황에 대한 데이터는 아주 풍부하게 얻을 수 있지만, 이상(Anomaly) 상황에 대한 데이터는 매우 부족할 뿐 아니라 사실상 모든 케이스의 이상 상황을 정의할 수조차 없기 때문입니다. 이 때 GAN이 사용된다면 이상(Anomaly) 데이터가 충분하지 않은 경우에도 꽤 쓸만한 이상감지 성능을 발휘하는 모델을 만들 수 있다고 합니다.

- 그럼 어떻게 이런 모델을 만들어 가는 것인지, 함께 알아보도록 할까요?

## 학습목표
- 이상(Anomaly) 데이터가 부족한 상황에서 GAN을 이용해 이미지 이상감지 모델을 구축하는 논리를 파악한다.
- Skip-GANomaly 모델 및 Loss 함수를 구현해 본다.
- 간단한 데이터셋을 이용해 Skip-GANomaly 의 이상감지 효과를 파악해 본다.

## 학습 목차
- Anomaly Detection with GAN
- GANomaly
- Skip-GANomaly
- 데이터셋 구성
- 모델과 Loss함수 구성
- 모델 학습과 평가

## 평가 루브릭

- 아래의 기준을 바탕으로 프로젝트를 평가합니다.

- 평가문항	상세기준
1. Anomaly Detection 태스크 특성에 맞도록 데이터셋 가공이 진행되었다.: 개구리 클래스가 배제되어 테스트셋에만 반영되는 로직이 구현되었다.
2. Skip-GANomaly 모델이 정상적으로 구현되었다.: 모델 학습 및 테스트가 원활하게 진행되었다.
3. 이상감지 수행 결과가 체계적으로 시각화되었다. : Anomaly score 분호, 테스트셋 이상감지 정확도(%), 이상감지 성공실패사례 제시가 진행되었다.

# Anomaly Detection with GAN

- 2017년에 발표된 Unsupervised Anomaly Detection with Generative Adversarial Networks to Guide Marker Discovery라는 논문에서 AnoGAN이라는 모델을 제안합니다. 논문의 내용은 제목에 그대로 담겨 있는데요, GAN을 이용하면 Anomaly 데이터가 부족한 상황에서도 Unsupervised 방식으로 Anomaly Detection 모델 구현이 가능하다는 아이디어입니다.

- 이게 어떻게 가능한 것일까요?

![title](E-28-01.max-800x600.png)

- 위 그림을 봅시다. 얼핏 보면 흔히 보는 GAN의 모델 구조를 가지고 있습니다. 여기서 중요한 것은 학습데이터로 Anomal하지 않은 정상(normal) 데이터만 사용한다는 점입니다. 그렇다면 GAN의 원리에 의해 Discriminator(빨간색)은 정상적인 이미지와 Generator(푸른색)만들어 낸 가짜 이미지를 잘 구분하게 될 것이고, Generator는 그런 Discriminator를 속이기 위해 정상 데이터같은 이미지를 만들어 낼 것입니다.

- 여기서 AnoGAN의 저자들은 이런 생각을 했습니다. 잘 훈련된 GAN의 Generator x=G(z)란 바로 Latent variable z와 이미지 x 사이의 어떤 맵핑을 나타내는 함수가 아닐까? 그렇다면 Normal 데이터를 잘 만들어낼 수 있는 z의 영역을 (z를 수없이 샘플링해 가면서) 더듬더듬 찾아내면 위 그림의 오른쪽과 같은 윤곽이 얻어지게 될 수 있지 않을까?

- 그래서 추가적인 Loss 2개를 아래와 같이 정의합니다.

![title](GANomaly.png)

- 그다음 아이디어는 어찌보면 당연한 것입니다. 지금까지 학습한 GAN의 Generator와 Discriminator는 모두 Normal 데이터에 대해서만 최적화된 파라미터를 가지고 있으며, Generator가 정상데이터처럼 보이도록 만드는 $ z_γ\\ $ 까지 찾아두었다면, Normal한 x에 대해서는 위 Loss값이 아주 작을 것입니다. 그러나 Anomaly 데이터 $ x $에 대해서는 이 Loss값이 일정 threshold 이상으로 치솟게 될 것이라는 논리입니다.